In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from datetime import datetime
import requests
from requests import get
import os
import time

In [3]:
#NASA LATEST MARS NEWS
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.body.find('div',class_='article_teaser_body').text
print(news_title)
print(news_p)

Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch
A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff.


In [4]:
#JPL Mars Space Images - Featured Image
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)
html_image = browser.html
soup = BeautifulSoup(html_image, 'html.parser')
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + featured_image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16726-1920x1200.jpg'

In [5]:
 #Mars Weather from twitter
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(mars_weather_url)
soup2 = BeautifulSoup(response.text, 'lxml')
 
#make empty list for weather tweets
weather_tweets = []

#iterate through soup and get tweet text
for li in soup2.find_all("li", {"data-item-type": "tweet"}):
    #make empty lists for all available tweets 
    all_tweets = []
    text_p = li.find("p", class_="tweet-text")
    if text_p is not None:
        all_tweets.append(text_p.get_text())
        for i in all_tweets: 
            t = []
            t.append(i)
            if any("sol" in s for s in t):
                weather_tweets.append(i)

#assign just current one to variable 
mars_current_weather = str(weather_tweets[0])
mars_current_weather

'InSight sol 528 (2020-05-22) low -93.5ºC (-136.3ºF) high -5.3ºC (22.4ºF)\nwinds from the SW at 6.3 m/s (14.0 mph) gusting to 23.6 m/s (52.8 mph)\npressure at 7.10 hPapic.twitter.com/bV5pLbAGoE'

In [6]:
#Mars Facts Information
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
mars_df=tables[0]
mars_df.columns=["description","value"]
mars_df.set_index("description",inplace=True)
mars_html_table=mars_df.to_html()
mars_html_table.replace('\n','')


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [7]:
#Mars Hemisphere images
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
hemi1=soup.find("div",class_='collapsible results')
hemisphere1=hemi1.find_all('a')
main_url="https://astrogeology.usgs.gov/"
hemisphere_image_urls=[]
for hemi in hemisphere1:
    if hemi.h3:
        title=hemi.h3.text
        link=hemi["href"]
        forward_url=main_url+link
        browser.visit(forward_url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        hemi2=soup.find("div",class_= "downloads")
        image=hemi2.ul.a["href"]
        hemi_dict={}
        hemi_dict["title"]=title
        hemi_dict["img_url"]=image
        hemisphere_image_urls.append(hemi_dict)
        browser.back()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [8]:
browser.quit()